接續方法2剩下7個位點 進行3個為一組組合 使用5種模型預測

In [ ]:
import pandas as pd
import xgboost as xgb

In [ ]:
normalized_train = pd.read_csv('../result/all_beta_normalized_train_0.35.csv')

normalized_test = pd.read_csv('../result/all_beta_normalized_test_0.35.csv')

normalized_train = normalized_train[normalized_train['Unnamed: 0'].isin(normalized_test['Unnamed: 0'])]

method2_gene=pd.read_csv("../result/overlap_select_gene_7.csv")
normalized_train_c= normalized_train[normalized_train['Unnamed: 0'].isin(method2_gene['ID'])]
normalized_train_c

In [ ]:
X_train = normalized_train_c.iloc[:, 1::2]

x = []

for i in range(445):
    X = X_train.iloc[:, i]

    X = X.values.flatten().tolist()

    x.append(X)

y = [ (0 if i < 47 else 1)  for i in range(445)]

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score
from itertools import combinations
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
target_labels = normalized_train_c['Unnamed: 0']
# 三個一組
combinations_list = list(combinations(target_labels, 3))

result=[]
# 定義模型參數
params = {
    'objective': 'binary:logistic',  # 二分類問題
    'eval_metric': 'logloss',  # 評估指標
    'max_depth': 3,
    'learning_rate': 1
}
models = {
    'XGBoost': xgb.XGBClassifier(**params),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='linear'),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():

    # 所有可能的組合
    for combination in combinations_list:
        #print(combination)
        d1_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[0]]
        d2_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[1]]
        d3_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[2]]
    
        d1 = d1_values.iloc[:, 1::2]
        d2 = d2_values.iloc[:, 1::2]
        d3 = d3_values.iloc[:, 1::2]

        d1 = d1.values.flatten().tolist()
        d2 = d2.values.flatten().tolist()
        d3 = d3.values.flatten().tolist()

        x = []

        for i in range(445):
            x.append([d1[i],d2[i],d3[i]])
            
        # 將數據轉換為DMatrix格式
        dmatrix = xgb.DMatrix(x, label=y)

        # 進行交叉驗證，這裡使用 5-fold 交叉驗證
        cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
        # 計算平均準確度
        average_accuracy = cv_scores.mean()

        # 獲取模型預測結果
        y_pred = cross_val_predict(model, x, y, cv=5)
        
        # 計算AUC
        auc_score = roc_auc_score(y, y_pred)
        
        # 計算F1 score
        f1 = f1_score(y, y_pred)
        GO_1=int(method2_gene.loc[method2_gene['ID'] == combination[0]]["GO"])
        GO_2=int(method2_gene.loc[method2_gene['ID'] == combination[1]]["GO"])
        GO_3=int(method2_gene.loc[method2_gene['ID'] == combination[2]]["GO"])
        if (GO_1!=GO_2 & GO_2!=GO_3 & GO_1!=GO_3):
            print(GO_1,GO_2,GO_3)
            if (GO_1==1 and GO_2==4 and GO_3==3):
                result.append([model_name, combination[0], combination[2], combination[1], average_accuracy, auc_score, f1])
            elif (GO_1==1 and GO_2==3 and GO_3==4):
                result.append([model_name, combination[0], combination[1], combination[2], average_accuracy, auc_score, f1])
            elif (GO_1==4 and GO_2==1 and GO_3==3):
                result.append([model_name, combination[1], combination[2], combination[0], average_accuracy, auc_score, f1])
            elif (GO_1==4 and GO_2==3 and GO_3==1):
                result.append([model_name, combination[2], combination[1], combination[0], average_accuracy, auc_score, f1])
            elif (GO_1==3 and GO_2==1 and GO_3==4):
                result.append([model_name, combination[1], combination[0], combination[2], average_accuracy, auc_score, f1])
            elif (GO_1==3 and GO_2==4 and GO_3==1):
                result.append([model_name, combination[2], combination[0], combination[1], average_accuracy, auc_score, f1])
            print(result)  
result


In [ ]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'ID3', 'average_accuracy','AUC','f1_score'])
df.to_csv("../result/ml_combination_method2.csv", index=False)

挑出3個為一組最佳的組合: MIR124-2、ZSCAN18、KCNE3 繪製normal/tumor間的關係

In [ ]:
d1_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg07792478']
d2_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg04574090']
d3_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg14231297']


print(d1_values)

d1 = d1_values.iloc[:, 1::2]
d2 = d2_values.iloc[:, 1::2]
d3 = d3_values.iloc[:, 1::2]

d1 = d1.values.flatten().tolist()
d2 = d2.values.flatten().tolist()
d3 = d3.values.flatten().tolist()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

def plot_3d_scatter(elev, azim):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(d1[:47], d2[:47], d3[:47], marker='o', color='blue', label='normal')
    ax.scatter(d1[47:], d2[47:], d3[47:], marker='x', color='red', label='tumor')

    ax.set_title('3D Scatter Plot')
    ax.set_xlabel('X-cg07792478')
    ax.set_ylabel('Y-cg04574090')
    ax.set_zlabel('Z-cg14231297')

    ax.legend()

    ax.view_init(elev=elev, azim=azim)
    
    plt.show()

interact(plot_3d_scatter, elev=(0, 90, 5), azim=(0, 360, 5))


接續方法1剩下40個位點進行GO分4群後實作RFECV總共剩下30個 於每群挑一個代表的位點 組合並預測

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score
from itertools import combinations
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
normalized_train = pd.read_csv('../result/all_beta_normalized_train_0.35.csv')

normalized_test = pd.read_csv('../result/all_beta_normalized_test_0.35.csv')

normalized_train = normalized_train[normalized_train['Unnamed: 0'].isin(normalized_test['Unnamed: 0'])]

cluster=pd.read_csv("../result/RFE_method1/RFECV/cluster.csv")
normalized_train_c= normalized_train[normalized_train['Unnamed: 0'].isin(cluster['ID'])]
normalized_train_c

In [ ]:
X_train = normalized_train_c.iloc[:, 1::2]

x = []

for i in range(445):
    X = X_train.iloc[:, i]

    X = X.values.flatten().tolist()

    x.append(X)

y = [ (0 if i < 47 else 1)  for i in range(445)]

In [ ]:
def is_unique(lst):
    return len(lst) == len(set(lst))

In [ ]:
target_labels = normalized_train_c['Unnamed: 0']
# 四個一組
combinations_list = list(combinations(target_labels, 4))

result=[]
# 定義模型參數
params = {
    'objective': 'binary:logistic',  # 二分類問題
    'eval_metric': 'logloss',  # 評估指標
    'max_depth': 3,
    'learning_rate': 1
}
models = {
    'XGBoost': xgb.XGBClassifier(**params),
    'RandomForest': RandomForestClassifier(n_estimators=100, 
                                            max_depth=10, 
                                            min_samples_split=5, 
                                            min_samples_leaf=2,
                                            min_impurity_decrease=0.0,
                                            random_state=42),
    'SVM': SVC(kernel='linear'),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():

    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(cluster.loc[cluster['ID'] == combination[0]]["GO"])
        GO_2=int(cluster.loc[cluster['ID'] == combination[1]]["GO"])
        GO_3=int(cluster.loc[cluster['ID'] == combination[2]]["GO"])
        GO_4=int(cluster.loc[cluster['ID'] == combination[3]]["GO"])

        lst=[GO_1,GO_2,GO_3,GO_4]

        if(is_unique(lst)):
            d1_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[0]]
            d2_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[1]]
            d3_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[2]]
            d4_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[3]]
        
            d1 = d1_values.iloc[:, 1::2]
            d2 = d2_values.iloc[:, 1::2]
            d3 = d3_values.iloc[:, 1::2]
            d4 = d4_values.iloc[:, 1::2]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            d3 = d3.values.flatten().tolist()
            d4 = d4.values.flatten().tolist()

            x = []

            for i in range(445):
                x.append([d1[i],d2[i],d3[i],d4[i]])
            

            dmatrix = xgb.DMatrix(x, label=y)

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1])
            
# result


In [ ]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'ID3','ID4', 'average_accuracy','AUC','f1_score'])
df.to_csv("../result/RFE_method1/RFECV/ml_combination_method1.csv", index=False)

In [ ]:
method1_combination=pd.read_csv("../result/RFE_method1/RFECV/ml_combination_method1.csv")

for i in range(0,len(method1_combination)-4,5):
    method1_combination['AVG_accuracy'][i]=(method1_combination['average_accuracy'][i:i+5]).mean()
    method1_combination['AVG_AUC'][i]=(method1_combination['AUC'][i:i+5]).mean()
    method1_combination['AVG_f1_score'][i]=(method1_combination['f1_score'][i:i+5]).mean()

In [ ]:
method1_combination.to_csv("../result/RFE_method1/RFECV/ml_combination_method1.csv", index=False)

挑選出4個為一組最佳組合:RGS22、SLC6A3、GHSR、ZSCAN18(和方法2相同)